## Combinatorial Optimization

#### Assignment IV

***Heuristics for the Knapsack Problem - Solving Multiple Instances***

**Instance:** C150I200

Guilherme Cadori

09/07/2023


In [1]:
# Importing required libraries
import numpy as np


#### 1. Read the instance

In [2]:
# Reading the target file
# GC: the code template below was provided in class and was adapted by GC (2023-07-09)

filename = "C:\gccadori\Lab4_CO\Knap_C150I200.dat"  # Substitua pelo nome do arquivo real

# Variáveis para armazenar os dados lidos
linha = None
coluna = None
cost = None
rhs = None
matrix = None

# Função auxiliar para converter uma linha de valores em uma lista de floats
def parse_values(line):
    return [float(value) for value in line.strip().split()]

# Ler o arquivo e extrair os dados
with open(filename, "r") as file:
    lines = file.readlines()

# Ler o escalar
linha = int(lines[0].strip())
coluna = int(lines[2].strip())

# Ler o vetor
cost = np.array(parse_values(lines[5]))

# Ler a matriz
end = 7 + linha
matrix_lines = [line.strip() for line in lines[7:end]]
matrix = np.array([parse_values(line) for line in matrix_lines],dtype=object)

# Ler vetor de RHS
rhs = np.array(parse_values(lines[end+1]))

# Imprimir os dados lidos
print("\n linhas:", linha)
print("\n colunas:", coluna)
print("\n cost:", cost)
print("\n rhs:", rhs)
print("\n Matrix:")
for row in matrix:
    print(row)



 linhas: 150

 colunas: 200

 cost: [0.30883 0.58904 0.96057 0.37599 0.32186 0.45693 0.60878 0.69994 0.90176
 0.79981 0.47232 0.28019 0.12491 0.39664 0.31095 0.15211 0.59066 0.24928
 0.73187 0.5131  0.68417 0.76533 0.92726 0.42301 0.53433 0.40482 0.7555
 0.74613 0.22319 0.19599 0.97708 0.74858 0.3439  0.91404 0.27935 0.95567
 0.93963 0.41099 0.47942 0.67846 0.80936 0.85349 0.56531 0.18551 0.94657
 0.97457 0.57162 0.15442 0.36188 0.1918  0.56081 0.49194 0.02246 0.49558
 0.2865  0.27893 0.94372 0.10526 0.13423 0.93888 0.73254 0.73492 0.74708
 0.18974 0.04297 0.12807 0.50676 0.18664 0.89319 0.85975 0.82796 0.49929
 0.58392 0.9172  0.46086 0.71906 0.31246 0.48658 0.96084 0.79158 0.02116
 0.69047 0.80266 0.24592 0.14197 0.04297 0.19352 0.55729 0.31395 0.58424
 0.39216 0.98363 0.93342 0.92955 0.87305 0.29755 0.94694 0.19158 0.84977
 0.0874  0.89123 0.90388 0.51277 0.18259 0.75101 0.1436  0.47817 0.67372
 0.16895 0.55436 0.07795 0.09752 0.08602 0.6577  0.93217 0.92999 0.31066
 0.20914 0.0859

#### 2.	Construct an initial feasible solution

In [3]:
# Renaming variables for testing purposes
c = cost

A = matrix

b = rhs


In [4]:
# Providing an initial feasible solution

def randomStart(nVars, A, b, c):
    # Creating variable names x1, ..., x_nVars
    variable_names = ['x{}'.format(i) for i in range(1, nVars+1)]

    while True:
        np.random.seed(12345)
        x = np.random.randint(2, size=nVars)
        
        if np.all(A @ x <= b):
            FO = x @ c
            print('The initial feasible solution is:')   
            print('Objective function value:', round(FO, 2))
            return


In [5]:
# Testing the function
randomStart(nVars = coluna, A = A, b = b, c = c)


The initial feasible solution is:
Objective function value: 53.1


#### 3.	Program the heuristic you proposed in the previous assingment

In [6]:
# Implementing the heuristic algorithm proposed in Lab III

def naiveKnapsack(nVars, items_c, capacity_b, A):
    # Sort the items in descending order of value
    sorted_items = sorted(range(nVars), key=lambda i: items_c[i], reverse=True)

    # Initialize variables
    selected_items = np.zeros(nVars, dtype=int)
    total_weight = np.zeros_like(capacity_b, dtype=items_c.dtype)
    total_value = 0

    # Greedy selection process
    for idx in sorted_items:
        weight = A[:, idx]
        if np.all(total_weight + weight <= capacity_b):
            # Add the item to the knapsack
            selected_items[idx] = 1
            total_weight += weight.astype(total_weight.dtype)  # Ensure matching data type
            total_value += items_c[idx]

    # Print the selected items and total value
    print('The initial feasible solution is:')
    print('Objective function value:', round(total_value, 2))

    # Return the complete list of items
    return


In [7]:
# Testing the function
naiveKnapsack(nVars = coluna, items_c = c, capacity_b = b , A = A)


The initial feasible solution is:
Objective function value: 89.63


#### 4.	Program a Local Search 

In [8]:
# Local Search metaheuristic implementation
def LocalSearch(nVars, A, b, c, nStarts=1, max_iterations=1000):
    import numpy as np

    # Creating variable names
    variable_names = ['x{}'.format(i) for i in range(1, nVars + 1)]

    best_solution = None
    best_value = -np.inf

    # Escaping Local Optima - Multiple Starting Solutions
    for n in range(1, nStarts + 1):
        # Producing an initial feasible solution
        while True:
            x = np.random.randint(2, size=nVars)
            if np.all(A @ x <= b):
                FO = x @ c
                break  # Exit the loop once an initial feasible solution is found

        # Setting initial search parameters
        s = FO
        s_new = 0
        n_iterations = 0
        best_iteration = 0

        # Termination criteria
        while n_iterations - best_iteration < max_iterations:
            n_iterations += 1

            # Neighbor Selection: Swap Two - First Improvement Strategy
            def generate_neighbor_solution(x):
                x_new = x.copy()
                idx1, idx2 = np.random.choice(len(x), size=2, replace=False)
                x_new[idx1], x_new[idx2] = x_new[idx2], x_new[idx1]
                return x_new

            # Generating neighbor solution
            x_new = generate_neighbor_solution(x)

            # Evaluate solution
            if np.all(A @ x_new <= b):
                FO_new = x_new @ c
            else:
                FO_new = -np.inf

            # Update the current solution
            if FO_new > s_new:
                x = x_new
                s_new = FO_new
                best_iteration = n_iterations

                # Check if the current solution is the best found so far
                if s_new > best_value:
                    best_solution = x.copy()
                    best_value = s_new

    # Print the best solution found
    print('The best solution found:')
    print('Objective function value:', round(best_value, 2))

    return


#### 5.	Running the heuristic for the given instance

In [23]:
import time

# Record starting time
start_time = time.time()

# Running the proposed heuristic for this instance
LocalSearch(nVars = coluna, A = A, b = b, c = c, 
            nStarts = 25,
            max_iterations = 150)

# Record ending time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = round(end_time - start_time, 2)

# Print the elapsed time
print("Elapsed time:", elapsed_time, "seconds")


The best solution found:
Objective function value: 83.37
Elapsed time: 63.4 seconds


**Test Run Results**

The best solution found:

- Objective function value: 83.37
- Elapsed time: 63.4 seconds


### End